In [1]:
import subprocess
from pyspark.sql.types import IntegerType
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import sum
from pyspark import SparkConf
from pyspark.sql import DataFrame,SparkSession


#Intializing SparkSession and configuration for spark.config can also set here 
conf = SparkConf().setMaster("local").setAppName("test")
spark = SparkSession.builder.config(conf=conf).getOrCreate()




In [2]:

# Function that is created to to read the hdfs file using SparkSession
def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True,inferSchema = True) 
    return rdd

#Function performs dataset computation and return dataframe for normalWeek and lockdownWeek  
def get_Dataframe(x):
    res = list()
    #This lines read the all names the file in hdfs folder and make it a single name with space seperated
    p=subprocess.Popen(x,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT)
    
    #then the line send to this for loop to run line by line files this has 7 data for normal week and 7 data for lockdownweek
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/data1.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/data2.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/Ldata1.csv
        #hdfs://localhost:9000/user/akash/trafficCount/aggre/Ldata1.csv
    for line in p.stdout.readlines():
        file = line.decode().strip()   
        vehicle_data= hdfs_file_read(file)
        
        output=vehicle_data.groupBy(vehicle_data.cosit,vehicle_data.day,vehicle_data.month,vehicle_data.year).sum('VehicleCount').sort('sum(VehicleCount)',ascending=False)
        output=output.withColumnRenamed("sum(VehicleCount)", "VehicleCount")
    #    output.show()
        output.createOrReplaceTempView('packages') 
        query_results = spark.sql('SELECT * FROM packages LIMIT 1') #This collects the first data 

    #append the results
        res.append(query_results)

    union_df = reduce(DataFrame.unionAll, res)
    return union_df


In [3]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownweek| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])
        
week0.show()#NormalWeek
week1.show()#LockdownWeek

+-----+---+-----+----+------------+
|cosit|day|month|year|VehicleCount|
+-----+---+-----+----+------------+
| 1502|  8|    4|2019|      156834|
| 1503|  9|    4|2019|      162944|
| 1503| 10|    4|2019|      166698|
| 1502| 11|    4|2019|      168417|
| 1502| 12|    4|2019|      171435|
| 1502| 13|    4|2019|      138231|
| 1502| 14|    4|2019|      120943|
+-----+---+-----+----+------------+

+-----+---+-----+----+------------+
|cosit|day|month|year|VehicleCount|
+-----+---+-----+----+------------+
| 1502|  6|    4|2020|       52110|
| 1502|  7|    4|2020|       53393|
| 1502|  8|    4|2020|       53123|
| 1502|  9|    4|2020|       52373|
| 1502| 10|    4|2020|       43681|
| 1502| 11|    4|2020|       24251|
| 1502| 12|    4|2020|       15321|
+-----+---+-----+----+------------+

